## Results of


MODEL_1 and 3 <br>
CLASSIFIED WITH WORDS and PHRASE<br>
ngram = 1 



In [1]:
from __future__ import print_function

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics

from sklearn.preprocessing import LabelEncoder
from nltk.stem.wordnet import WordNetLemmatizer
import re
import itertools
import os.path
import json
from datetime import datetime

import pandas as pd


In [2]:
def remove_numbers(ing):
    # remove numbers from ingredients
    
    return [[re.sub("\d+", "", x) for x in y] for y in ing]

    
def remove_special_chars(ing):
    # remove certain special characters from ingredients
   
    ing = [[x.replace("-", " ") for x in y] for y in ing] 
    ing = [[x.replace("&", " ") for x in y] for y in ing] 
    ing = [[x.replace("'", " ") for x in y] for y in ing] 
    ing = [[x.replace("''", " ") for x in y] for y in ing] 
    ing = [[x.replace("%", " ") for x in y] for y in ing] 
    ing = [[x.replace("!", " ") for x in y] for y in ing] 
    ing = [[x.replace("(", " ") for x in y] for y in ing] 
    ing = [[x.replace(")", " ") for x in y] for y in ing] 
    ing = [[x.replace("/", " ") for x in y] for y in ing] 
    ing = [[x.replace("/", " ") for x in y] for y in ing] 
    ing = [[x.replace(",", " ") for x in y] for y in ing] 
    ing = [[x.replace(".", " ") for x in y] for y in ing] 
    ing = [[x.replace(u"\u2122", " ") for x in y] for y in ing] 
    ing = [[x.replace(u"\u00AE", " ") for x in y] for y in ing] 
    ing = [[x.replace(u"\u2019", " ") for x in y] for y in ing] 

    return ing
    
    
def make_lowercase(ing):
    # make all letters lowercase for all ingredients
    
    return [[x.lower() for x in y] for y in ing]
    
    
def remove_extra_whitespace(ing):
    # removes extra whitespaces
    
    return [[re.sub( '\s+', ' ', x).strip() for x in y] for y in ing] 
    
    
def stem_words(ing):
    # word stemming for ingredients
    
    lmtzr = WordNetLemmatizer()
    
    def word_by_word(strng):
        
        return " ".join(["".join(lmtzr.lemmatize(w)) for w in strng.split()])
    
    return [[word_by_word(x) for x in y] for y in ing] 
    
    
def remove_units(ing):
    # remove certain words from ingredients
    
    remove_list = ['g', 'lb', 's', 'n']
        
    def check_word(strng):
        
        s = strng.split()
        resw  = [word for word in s if word.lower() not in remove_list]
        
        return ' '.join(resw)

    return [[check_word(x) for x in y] for y in ing] 


In [3]:
### by all the words

df=pd.read_json('./train.json')
X = df['ingredients'].values
Y = df['cuisine'].values

In [4]:
X = make_lowercase(X)
X = remove_numbers(X)
X = remove_special_chars(X)
X = remove_extra_whitespace(X)
X = remove_units(X)
X = stem_words(X)

In [5]:
df['ingredients_preprocessed'] = X

In [6]:
df['ingredients_as_sentence'] = df['ingredients_preprocessed'].apply(', '.join)

In [7]:
uniques = list(set([item for sublist in X for item in sublist]))
print(len(uniques))

6677


In [8]:
X = df['ingredients_as_sentence'].values

In [9]:
for ing in uniques:
    temp = ing.split(' ')
    for word in temp:
        if word not in uniques:
            uniques.append(word)
print(len(uniques))

8285


In [10]:
ngram = 1     #### CUSTOMIZE 

# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, ngram), vocabulary=uniques)
X = vectorizer.fit_transform(X)

In [14]:
print(X[0])

  (0, 7434)	0.4043455766076157
  (0, 7412)	0.3167276767158082
  (0, 7058)	0.24911439037394897
  (0, 6765)	0.35697121618985855
  (0, 6533)	0.14805335445131518
  (0, 5445)	0.10970675831861157
  (0, 4551)	0.2774555999091271
  (0, 4306)	0.15176502988889437
  (0, 3811)	0.2396832355219728
  (0, 3570)	0.1118145095283498
  (0, 3272)	0.10391647275486823
  (0, 3094)	0.3560443893695225
  (0, 2952)	0.1457359419289999
  (0, 2283)	0.34827614404150986
  (0, 1263)	0.20790510180038077
  (0, 122)	0.13948352921322324


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2 ,random_state=2019, stratify = Y )
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(31819, 8285) (7955, 8285) (31819,) (7955,)


In [15]:
def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(y_test, pred)
    print("accuracy:   %0.3f" % score)
    
    
    if hasattr(clf, 'coef_'):
        print("dimensionality: %d" % clf.coef_.shape[1])
        print("density: %f" % density(clf.coef_))
        
    print("classification report:")
    print(metrics.classification_report(y_test, pred,target_names=target_names))
    
    
    #if opts.print_cm:
    #print("confusion matrix:")
    #print(metrics.confusion_matrix(y_test, pred))

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, train_time, test_time


In [16]:
RANDOM_ST = 2019

target_names = ['brazilian', 'british', 'cajun_creole', 'chinese', 'filipino', 'french', 'greek',
           'indian', 'irish', 'italian', 'jamaican', 'japanese', 'korean', 'mexican', 'moroccan',
           'russian', 'southern_us', 'spanish', 'thai', 'vietnamese']

results = []
for clf, name in (
        (RidgeClassifier(tol=1e-2, solver="sag",random_state=RANDOM_ST), "Ridge Classifier"),
        (Perceptron(max_iter=50, tol=1e-3,random_state=RANDOM_ST), "Perceptron"),
        (PassiveAggressiveClassifier(max_iter=50, tol=1e-3,random_state=RANDOM_ST), "Passive-Aggressive"),
        (KNeighborsClassifier(n_neighbors=10), "kNN"),
        (RandomForestClassifier(n_estimators=100,random_state=RANDOM_ST), "Random forest")):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf))
    
for penalty in ["l2", "l1"]:
    print('=' * 80)
    print("%s penalty" % penalty.upper())
    # Train Liblinear model
    results.append(benchmark(LinearSVC(penalty=penalty, dual=False, tol=1e-3)))

    # Train SGD model
    results.append(benchmark(SGDClassifier(alpha=.0001, max_iter=50,penalty=penalty,random_state=RANDOM_ST)))

# Train SGD with Elastic Net penalty
print('=' * 80)
print("Elastic-Net penalty")
results.append(benchmark(SGDClassifier(alpha=.0001, max_iter=50,penalty="elasticnet",random_state=RANDOM_ST)))

# Train NearestCentroid without threshold
print('=' * 80)
print("NearestCentroid (aka Rocchio classifier)")
results.append(benchmark(NearestCentroid()))

# Train sparse Naive Bayes classifiers
print('=' * 80)
print("Naive Bayes")
results.append(benchmark(MultinomialNB(alpha=.01)))
results.append(benchmark(BernoulliNB(alpha=.01)))
results.append(benchmark(ComplementNB(alpha=.1)))

print('=' * 80)
print("LinearSVC with L1-based feature selection")
# The smaller C, the stronger the regularization.
# The more regularization, the more sparsity.
results.append(benchmark(Pipeline([('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False,tol=1e-3,random_state=RANDOM_ST))),
  ('classification', LinearSVC(penalty="l2",random_state=RANDOM_ST))])))

Ridge Classifier
________________________________________________________________________________
Training: 
RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=2019, solver='sag',
        tol=0.01)
train time: 2.699s
test time:  0.003s
accuracy:   0.764
dimensionality: 8285
density: 0.322028
classification report:
              precision    recall  f1-score   support

   brazilian       0.75      0.43      0.55        93
     british       0.74      0.30      0.43       161
cajun_creole       0.79      0.65      0.72       309
     chinese       0.72      0.88      0.80       535
    filipino       0.73      0.43      0.54       151
      french       0.61      0.66      0.63       529
       greek       0.82      0.68      0.74       235
      indian       0.83      0.89      0.86       601
       irish       0.69      0.29      0.40       133
     italian       0.76      0.91      0.83      1568
    jama

train time: 1.914s
test time:  0.003s
accuracy:   0.796
dimensionality: 8285
density: 0.322028
classification report:
              precision    recall  f1-score   support

   brazilian       0.77      0.59      0.67        93
     british       0.68      0.46      0.55       161
cajun_creole       0.76      0.72      0.74       309
     chinese       0.80      0.86      0.83       535
    filipino       0.76      0.62      0.68       151
      french       0.62      0.69      0.65       529
       greek       0.81      0.72      0.77       235
      indian       0.88      0.91      0.89       601
       irish       0.65      0.45      0.53       133
     italian       0.83      0.88      0.86      1568
    jamaican       0.82      0.71      0.76       105
    japanese       0.84      0.70      0.77       284
      korean       0.82      0.80      0.81       166
     mexican       0.90      0.93      0.91      1288
    moroccan       0.80      0.79      0.79       164
     russian     

/home/ncp/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


train time: 2.473s
test time:  0.003s
accuracy:   0.781
dimensionality: 8285
density: 0.150398
classification report:
              precision    recall  f1-score   support

   brazilian       0.74      0.49      0.59        93
     british       0.69      0.27      0.39       161
cajun_creole       0.76      0.70      0.73       309
     chinese       0.78      0.88      0.83       535
    filipino       0.74      0.50      0.60       151
      french       0.65      0.63      0.64       529
       greek       0.82      0.71      0.76       235
      indian       0.83      0.91      0.87       601
       irish       0.72      0.32      0.45       133
     italian       0.79      0.91      0.85      1568
    jamaican       0.82      0.68      0.74       105
    japanese       0.82      0.68      0.74       284
      korean       0.79      0.72      0.75       166
     mexican       0.89      0.93      0.91      1288
    moroccan       0.80      0.76      0.78       164
     russian     

              precision    recall  f1-score   support

   brazilian       0.81      0.45      0.58        93
     british       0.53      0.30      0.39       161
cajun_creole       0.69      0.64      0.66       309
     chinese       0.72      0.88      0.80       535
    filipino       0.79      0.47      0.59       151
      french       0.57      0.57      0.57       529
       greek       0.87      0.54      0.67       235
      indian       0.83      0.89      0.86       601
       irish       0.71      0.26      0.38       133
     italian       0.75      0.88      0.81      1568
    jamaican       0.88      0.50      0.63       105
    japanese       0.89      0.62      0.73       284
      korean       0.87      0.62      0.73       166
     mexican       0.86      0.89      0.88      1288
    moroccan       0.81      0.66      0.73       164
     russian       0.68      0.31      0.42        98
 southern_us       0.57      0.75      0.65       864
     spanish       0.72    

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import xgboost as xgb

# xgb  <- xgboost(xgbmat, max.depth = 25, 
# eta = 0.3, nround = 200, objective = "multi:softmax", num_class = 20)
# https://www.kaggle.com/mohdatir/xgboost

results.append(benchmark(xgb.XGBClassifier(max_depth=25, n_estimators=400, random_state=RANDOM_ST)))

# objective='binary:logistic'  objective issue : 
# https://stackoverflow.com/questions/39386966/multiclass-classification-in-xgboost-python


________________________________________________________________________________
Training: 
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=25, min_child_weight=1, missing=None, n_estimators=400,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=2019, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)
train time: 2393.403s
test time:  20.284s
accuracy:   0.791
classification report:
              precision    recall  f1-score   support

   brazilian       0.86      0.60      0.71        93
     british       0.74      0.42      0.54       161
cajun_creole       0.79      0.69      0.74       309
     chinese       0.80      0.86      0.83       535
    filipino       0.71      0.58      0.64       151
      french       0.62      0.64      0.63       529
       greek       0.81      0.70      0.75       235


In [ ]:
print(results)